# Spectrophotometer write-up

## Section 1: Overview

The goal of the project wwas to build a spectrophotometer and obtain an example growth curve. The spectrophotometer was built from the components in the kit and some household items like an empty protein bar box, a black straw, and tape. The spectrophometer was to operate in stand-alone mode, in which the device does not need to be connected to a computer (only a power source). The user can toggle a switch to turn on the LED and the resulting OD is displayed on the LCD display. It was also to be operated while controlled by a computer. In this mode, the user has an on-screen control panel and can click buttons to obtain and display OD measurements.

Once the spectrophotometer was set up, we were to demonstrate the instrument by acquiring a growth curve and determining the growth rate 𝑘 of yeast in broth medium.

## Section 2: Design and design considerations

The schematic and pictorial representation we will use is shown below.

![Spectro_schem.png](Spectro_schem.png)
![Pictorial.png](Pictorial.png)

### Breadboard set up

**Light source**: First I choose a 600nm LED and connected the cathode to ground. The anode is connected to one of the outer pins of the toggle switch. I chose to use a toggle switch so I can control when the LED is turned on, so as to not burn out the brightness and intensity. Then I choose to use a $220\Omega$ resistor with the LED so that it does not burn out over time.

**Sensitivity to light source**: I chose to use a photoresistor to detect the light coming out of the LED and through the sample. I chose to use a voltage divider consisting of the photoresistor and a $10K\Omega$ resistor since that will give us a large dynamic range of voltages, with respective sensitivity to light.

**Including MCP4725 DAC**: I chose to include this breakout board for 1) ease of assembly with the LCD and previous code supplied in the lessons and 2) for potential adjustments or further experiments where digital-to-analog conversion may be useful.

**Using an LCD**: I used the "backpack" of the LCD to power the LCD and connect it to Arduino using I2C using SDA and SCL. I used the LCD to display voltages while I was performing my calibration experiment with an ink source and the cellophane. Subsequently, I used the LCD to continuously display OD while plugged into a power source. Similar to the follow along, I display OD as well as the graphical display of OD on the second row of the LCD.

### Spectrophotometer prototype set up

![Spec](Spec_pic.jpg)

**Cardboard box**: I used a old cardboard to perform the spectrophotometer measurements. I used scissors to cut an opening at the appropriate angle of incidence. I then used a pencil to open the hole large enough for the straw to fit snug. I used tape to stabilize the prototype as much as possible.

**Stray light consideration**: I could block stray light by using black straws as my apertures. 

**Collimating the light**: I chose to collimate light by using an aperture designed from black straws, allowing the light the travel through the holes with minimal scattering. The light collimates through the first straw and passes through before reaching the cuvette. Then after hitting the cuvette, the light will scatter so the remaining light is captured by the straw again such that light is directly fed to the photoresistor.

### User interface

Find comments alongside the code below for user interface design considerations.

### Potential limitations or trade offs for major design decisions

- **Spectrophotometer stability**: using a cardboard box is not the most reliable set up. The angles and wholes were mostly aligned, but still a little off. Therefore, not all the light leaving the LED will hit the photoresistor directly. However, as long as the amount of light hitting the photoresistor is consistent across the blank and the sample measurements, then the adjusted OD should account for the limitation.

- **Dimming LED**: over time, if the LED is on for long bouts of time, the intensity will begin to decrease. To combat this, I added a toggle switch to turn on and off the LED. This will minimize the amount of time that the LED is turned on.

- **LCD is always on**: I chose to leave the LCD on at all times that the Arduino is plugged into power. In this way, OD is being measured at all times. 

## Section 3: Instructions for use

Please find PDF in ZIP folder for instructions.

## Section 4: Demonstration and assessment

Find below the Arduino sketch that works for the standalone version and the computer version to run the spectrophotometer.

```cpp
#include <Adafruit_MCP4725.h>
#include <LiquidCrystal_I2C.h>
#include <Wire.h>

// This is the I2C Address of the MCP4725, by default (A0 pulled to GND).
// For devices with A0 pulled HIGH, use 0x61
#define MCP4725_ADDR 0x62

// LCD address
#define LCD_ADDR 0x27

// Geometry of LCD
const int nRows = 2;
const int nCols = 16;

// Custom characters for LCD bars
const byte zeroBlock[8] = {B00000, B00000, B00000, B00000, B00000, B00000, B00000, B00000};
const byte twentyBlock[8] = {B10000, B10000, B10000, B10000, B10000, B10000, B10000, B10000};
const byte fortyBlock[8] = {B11000, B11000, B11000, B11000, B11000, B11000, B11000, B11000};
const byte sixtyBlock[8] = {B11100, B11100, B11100, B11100, B11100, B11100, B11100, B11100};
const byte eightyBlock[8] = {B11110, B11110, B11110, B11110, B11110, B11110, B11110, B11110};
const byte fullBlock[8] = {B11111, B11111, B11111, B11111, B11111, B11111, B11111, B11111};

// Frequency of oscillating signal
const int freq = 1;

// Delays for updating
const unsigned long sampleDelay = 20;
unsigned long lastSampleTime = 0;
const unsigned long reportDelay = 100;
unsigned long lastReportTime = 0;

// Pin connected to voltage divider
const int voltagePin = A0;

const int HANDSHAKE = 0;
const int VOLTAGE_REQUEST = 1;
const int ON_REQUEST = 2;
const int STREAM = 3;
const int READ_DAQ_DELAY = 4;

// Initially, only send data upon request
int daqMode = ON_REQUEST;

// Default time between data acquisition is 100 ms
int daqDelay = 100;

// String to store input of DAQ delay
String daqDelayStr;


// Keep track of last data acquistion for delays
unsigned long timeOfLastDAQ = 0;

unsigned long printVoltage() {
  // Read value from analog pin
  int value = analogRead(voltagePin);
  
  float volts = value * 5 / 1023;

  float OD = 1.8226 * exp(-0.678 * volts);

  // Get the time point
  unsigned long timeMilliseconds = millis();

  // Write the result
  if (Serial.availableForWrite()) {
    String outstr = String(String(timeMilliseconds, DEC) + "," 
    + String(value, DEC));
    Serial.println(outstr);
  }

  // Return time of acquisition
  return timeMilliseconds;
}


// Instantiate the convenient classses for DAC and LCD
Adafruit_MCP4725 dac;
LiquidCrystal_I2C lcd = LiquidCrystal_I2C(LCD_ADDR, nCols, nRows);

void writeGraphicalOD(int x, int minX, int maxX) {
  /*
   * Graphical display of voltage on second row of LCD
   */
  float frac = ((float) (x - minX)) / (maxX - minX);
  int nBars = (int) (nCols * 5 * frac);
  int n5BarBlocks = nBars / 5;
  int fracBlock = nBars % 5;

  // Write the parts that are full blocks
  for (int i = 0; i < n5BarBlocks; i++) {
    lcd.setCursor(i, 1);
    lcd.write(5);
  }

  // Write fractional block
  if (n5BarBlocks < nCols) {
    lcd.setCursor(n5BarBlocks, 1);
    lcd.write(fracBlock);
  }

  // Write blank blocks
  for (int i = n5BarBlocks + 1; i < nCols; i++) {
    lcd.setCursor(i, 1);
    lcd.write(0);
  }
}


void setup() {
  // Set I2C to be fast mode
  Wire.setClock(400000);

  // Initialize serial communication
  Serial.begin(115200);

  dac.begin(MCP4725_ADDR);

  // Initialize the LCD
  lcd.init();
  lcd.backlight();

  // Add the special characters for blocks
  lcd.createChar(0, zeroBlock);
  lcd.createChar(1, twentyBlock);
  lcd.createChar(2, fortyBlock);
  lcd.createChar(3, sixtyBlock);
  lcd.createChar(4, eightyBlock);
  lcd.createChar(5, fullBlock);

  // Label message on LCD
  lcd.setCursor(0, 0);
  lcd.print("OD: ");
}


void loop() {
  // If we're streaming
  if (daqMode == STREAM) {
    if (millis() - timeOfLastDAQ >= daqDelay) {
      timeOfLastDAQ = printVoltage();
    }
  }

  // Check if data has been sent to Arduino and respond accordingly
  if (Serial.available() > 0) {
    // Read in request
    int inByte = Serial.read();

    // If data is requested, fetch it and write it, or handshake
    switch(inByte) {
      case VOLTAGE_REQUEST:
        timeOfLastDAQ = printVoltage();
        break;
      case ON_REQUEST:
        daqMode = ON_REQUEST;
        break;
      case STREAM:
        daqMode = STREAM;
        break;
      case READ_DAQ_DELAY:
        // Read in delay, knowing it is appended with an x
        daqDelayStr = Serial.readStringUntil('x');

        // Convert to int and store
        daqDelay = daqDelayStr.toInt();

        break;
      case HANDSHAKE:
        if (Serial.availableForWrite()) {
          Serial.println("Message received.");
        }
        break;
    }
  }

        
  unsigned long currTime = millis();

  if (currTime - lastSampleTime > sampleDelay) {
    // Sinusoidal signal to DAC
    uint16_t x = (uint16_t)(4095 * (1 + sin(2 * PI * freq * millis() / 1000.0)) / 2.0);

    dac.setVoltage(x, false);

    lastSampleTime = currTime;

    // Report voltage to LCE
    if (currTime - lastReportTime > reportDelay) {
      // Read in and convert voltage
      int sensorValue = analogRead(voltagePin);
      float voltage = sensorValue / 1023.0 * 5.0;
      float OD = 1.8226 * exp(-0.678 * voltage);

      // Write the voltage on the first row (text display)
      lcd.setCursor(9, 0);
      lcd.print(String(OD, 2));

      // Display the voltage graphically
      writeGraphicalOD(OD, 0, 1.82);

      lastReportTime = currTime;
      //Serial.print(voltage);
      //Serial.print(" ");
      //Serial.println(OD);
    }

  }

}
````

In [1]:
import asyncio
import re
import sys
import time

import numpy as np
import pandas as pd

import serial
import serial.tools.list_ports

import bokeh.plotting
import bokeh.io
import bokeh.layouts
import bokeh.driving
bokeh.io.output_notebook()

notebook_url = "localhost:8888"

Loading BokehJS ...

Run the utility functions below. Note that *parse_read()* and *parse_raw()* now return **time, od** instead of **time, voltage**. As a result, *request_single_voltage()* will return a single point **time, od**.

In [2]:
def find_arduino(port=None):
    """Get the name of the port that is connected to Arduino."""
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
    return port


def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=0
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout


def read_all(ser, read_buffer=b"", **args):
    """Read all available bytes from the serial port
    and append to the read buffer.

    Parameters
    ----------
    ser : serial.Serial() instance
        The device we are reading from.
    read_buffer : bytes, default b''
        Previous read buffer that is appended to.

    Returns
    -------
    output : bytes
        Bytes object that contains read_buffer + read.

    Notes
    -----
    .. `**args` appears, but is never used. This is for
       compatibility with `read_all_newlines()` as a
       drop-in replacement for this function.
    """
    # Set timeout to None to make sure we read all bytes
    previous_timeout = ser.timeout
    ser.timeout = None

    in_waiting = ser.in_waiting
    read = ser.read(size=in_waiting)

    # Reset to previous timeout
    ser.timeout = previous_timeout

    return read_buffer + read


def read_all_newlines(ser, read_buffer=b"", n_reads=4):
    """Read data in until encountering newlines.

    Parameters
    ----------
    ser : serial.Serial() instance
        The device we are reading from.
    n_reads : int
        The number of reads up to newlines
    read_buffer : bytes, default b''
        Previous read buffer that is appended to.

    Returns
    -------
    output : bytes
        Bytes object that contains read_buffer + read.

    Notes
    -----
    .. This is a drop-in replacement for read_all().
    """
    raw = read_buffer
    for _ in range(n_reads):
        raw += ser.read_until()

    return raw


def parse_read(read):
    """Parse a read with time, volage data

    Parameters
    ----------
    read : byte string
        Byte string with comma delimited time/voltage
        measurements.

    Returns
    -------
    time_ms : list of ints
        Time points in milliseconds.
    od : list of floats
        optical density.
    remaining_bytes : byte string
        Remaining, unparsed bytes.
    """
    time_ms = []
    od = []

    # Separate independent time/voltage measurements
    pattern = re.compile(b"\d+|,")
    raw_list = [
        b"".join(pattern.findall(raw)).decode()
        for raw in read.split(b"\r\n")
    ]

    for raw in raw_list[:-1]:
        try:
            t, V = raw.split(",")
            time_ms.append(int(t))
            # Use calibration equation to return value for OD
            od.append(float(1.8226 * np.exp((int(V) * 5 / 1023) * -0.678)))
        except:
            pass

    if len(raw_list) == 0:
        return time_ms, od, b""
    else:
        return time_ms, od, raw_list[-1].encode()


def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    t, V = raw.rstrip().split(",")
    
    # Use calibration equation to return value for OD

    return int(t), float(1.8226 * np.exp((int(V) * 5 / 1023) * -0.678))


def request_single_voltage(arduino):
    """Ask Arduino for a single data point"""
    # Ask Arduino for data
    arduino.write(bytes([VOLTAGE_REQUEST]))

    # Read in the data
    raw = arduino.read_until()

    # Parse and return
    return parse_raw(raw)

### Demonstration of computer mode

First, we open a connection to Arduino.

In [4]:
# Set up connection
HANDSHAKE = 0
VOLTAGE_REQUEST = 1
ON_REQUEST = 2;
STREAM = 3;
READ_DAQ_DELAY = 4;

port = find_arduino()
arduino = serial.Serial(port, baudrate=115200)
handshake_arduino(arduino)

Next, we store data coming in from the Arduino into data collection functions' using dictionaries of lists, one for each OD data, growth curve data, and your blank cuvette data.

In [5]:
# Set up data dictionaries
stream_data = dict(prev_array_length=0, t=[], OD=[], mode='on demand')
on_demand_data = dict(t=[], OD=[])
growth_data = dict(t=[],lnOD=[])
blank_data = dict(t=[], OD=[])

In [6]:
async def daq_stream_async(
    arduino, data, delay=20, n_trash_reads=5, n_reads_per_chunk=4, reader=read_all_newlines
):
    """Obtain streaming data"""
    # Specify delay
    arduino.write(bytes([READ_DAQ_DELAY]) + (str(delay) + "x").encode())

    # Current streaming state
    stream_on = False

    # Receive data
    read_buffer = [b""]
    while True:
        if data["mode"] == "stream":
            # Turn on the stream if need be
            if not stream_on:
                arduino.write(bytes([STREAM]))

                # Read and throw out first few reads
                i = 0
                while i < n_trash_reads:
                    _ = arduino.read_until()
                    i += 1

                stream_on = True

            # Read in chunk of data
            raw = reader(
                arduino, read_buffer=read_buffer[0], n_reads=n_reads_per_chunk
            )

            # Parse it, passing if it is gibberish
            try:
                t, OD, read_buffer[0] = parse_read(raw)

                # Update data dictionary
                data["t"] += t
                data["OD"] += OD
            except:
                pass
        else:
            # Make sure stream is off
            stream_on = False

        # Sleep 80% of the time before we need to start reading chunks
        await asyncio.sleep(0.8 * n_reads_per_chunk * delay / 1000)


daq_task = asyncio.create_task(daq_stream_async(arduino, stream_data))

First, we build the plot that Bokeh will use to create the app. One plot is built for optical density readings and the other is built to show the growth curve, using ln(OD600) on the y axis.

In [7]:
def plot(mode):
    """Build a plot of optical density vs time data"""
    # Set up plot area
    p = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (s)",
        y_axis_label="optical density (OD)",
        title="Optical density measurements",
        y_range=[-0.2, 2.5],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p.border_fill_color = "whitesmoke"

    # Defined the data source
    source = bokeh.models.ColumnDataSource(data=dict(t=[], OD=[]))

    # Use dots for on-demand
    p.circle(source=source, x="t", y="OD")

    # Put a phantom circle so axis labels show before data arrive
    phantom_source = bokeh.models.ColumnDataSource(data=dict(t=[0], OD=[0]))
    p.circle(source=phantom_source, x="t", y="OD", visible=False)
    
    """Build a plot of ln(optical density) vs time data"""
    # Set up plot area
    p1 = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (h)",
        y_axis_label="lnOD600 (OD)",
        title="Growth curve",
        y_range=[-6.0, 1.5],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p1.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p1.border_fill_color = "whitesmoke"

    # Defined the data source
    source1 = bokeh.models.ColumnDataSource(data=dict(t=[], lnOD=[]))

    # Use dots for on-demand
    p1.circle(source=source1, x="t", y="lnOD")

    # Put a phantom circle so axis labels show before data arrive
    phantom_source1 = bokeh.models.ColumnDataSource(data=dict(t=[0], lnOD=[0]))
    p1.circle(source=phantom_source1, x="t", y="lnOD", visible=False)

    return p, source, phantom_source, p1, source1, phantom_source1 

Then we build and store controls for the app.

In [8]:
def controls():
    acquire = bokeh.models.Button(label="acquire sample measurement", button_type="success", width=100)
    save_notice = bokeh.models.Div(
            text="<p>No on-demand data saved.</p>", width=165
        )

    # button to measure OD of blank
    blank = bokeh.models.Button(label="measure blank", button_type="primary", width=100)
    # Text to show blank OD
    display = bokeh.models.Div(text="""Measure blank""", width=150, height=25)
    
    save = bokeh.models.Button(label="save", button_type="primary", width=100)
    reset_blank = bokeh.models.Button(label="reset blank", button_type="danger", width=100)
    reset_data = bokeh.models.Button(label="reset data", button_type="danger", width=100)
    file_input = bokeh.models.TextInput(
        title="file name", value=f"spectrophotometer_experiment.csv", width=250
    )

    return dict(
        acquire=acquire,
        blank=blank,
        display=display,
        reset_blank=reset_blank,
        reset_data=reset_data,
        save=save,
        file_input=file_input,
        save_notice=save_notice,
    )

We lay out the plots and controls.

In [9]:
def layout(p, p1, ctrls):
    buttons = bokeh.layouts.row(
        bokeh.models.Spacer(width=30),
        ctrls["blank"],
        ctrls["display"],
        ctrls["reset_blank"],
        ctrls["acquire"],
        bokeh.models.Spacer(width=80),
        ctrls["reset_data"],
    )
    left = bokeh.layouts.column(p, buttons, p1, spacing=15)
    right = bokeh.layouts.column(
        bokeh.models.Spacer(height=50),
        ctrls["file_input"],
        ctrls["save"],
        ctrls["save_notice"],
        bokeh.models.Spacer(height=20)
    )
    return bokeh.layouts.row(
        left, right, spacing=30, margin=(30, 30, 30, 30),
    )

def layout_growth(p, ctrls):
    buttons = bokeh.layouts.row(
        bokeh.models.Spacer(width=30),
        ctrls["growth_curve"]
    )
    lay = bokeh.layouts.column(p, buttons, spacing=15)
    
    return bokeh.layouts.row(lay, margin=(30, 30, 30, 30))

Then we write callbacks so that clicking buttons will execute the desired functions. First we write a callback function to calibrate the experiment by updating the *blank_data* dictionary with a value from the blank cuvette. We also write a function to clear the *blank_data* dictionary.

In [11]:
def blank_callback(arduino, controls):
    # Pull t and OD value from request from Arduino
    t, OD = request_single_voltage(arduino)
    
    # Add to blank data dictionary
    blank_data["t"].append(t)
    blank_data["OD"].append(OD)
    
    controls["display"].text = 'OD = {}'.format(OD)

def reset_blank_callback(arduino, controls):
    blank_data["t"] = []
    blank_data["OD"] = []
    
    controls["display"].text = "Measure blank"

This callback function requests a time and OD from the Arduino and then adjusts the OD based on the last measurement in *blank_data*. Then it appends to *on_demand_data* as **time (seconds), adjusted OD**. It also appends to *growth_data* as **time (hours), ln(adjusted OD)**.

In [12]:
def acquire_callback(arduino, blank_data, source, phantom_source, source1, phantom_source1, rollover):
    # Pull t and OD values from request from Arduino
    t, OD = request_single_voltage(arduino)
    OD = OD - blank_data["OD"][-1]
    lnOD = np.log(OD)

    # Add to on-demand data dictionary
    on_demand_data["t"].append(t/1000)
    on_demand_data["OD"].append(OD)
    
    # Add to growth data dictionary
    growth_data["t"].append(t/3600000)
    growth_data["lnOD"].append(lnOD)

    # Send new data to plot
    new_data = dict(t=[t/1000], OD=[OD])
    source.stream(new_data, rollover=rollover)
    
    new_data_growth = dict(t=[t/3600000], lnOD=[lnOD])
    source1.stream(new_data_growth, rollover=rollover)

    # Update the phantom source to keep the x_range on plot ok
    phantom_source.data = new_data
    phantom_source1.data = new_data_growth

The *reset_data_callback* will reset the measurement data by clearing all of the arrays and sources holding data.

In [13]:
def reset_data_callback(data, data1, source, source1, phantom_source, phantom_source1, controls):
    # Black out the data dictionaries
    data["t"] = []
    data["OD"] = []
    data1["t"] = []
    data1["lnOD"] = []

    # Reset the sources
    source.data = dict(t=[], OD=[])
    phantom_source.data = dict(t=[0], OD=[0])
    source1.data = dict(t=[], lnOD=[])
    phantom_source1.data = dict(t=[0], lnOD=[0])

The *save_callback* function will take the time, OD, and ln(OD), put them in a Pandas data frame, and write the results to the CSV file.

In [14]:
def save_callback(data, data1, controls):
    # Convert data to data frame and save
    df = pd.DataFrame(data={"time (h)": data1["t"], "optical density (OD)": data["OD"], "lnOD600": data1["lnOD"]})
    df.to_csv(controls["file_input"].value, index=False)

    # Update notice text
    notice_text = "<p>" + ("On-demand")
    notice_text += f" data was last saved to {controls['file_input'].value}.</p>"
    controls["save_notice"].text = notice_text

The *shutdown_callback* will disable all controls and close the connection to Arduino. It will kill the app.

In [15]:
def disable_controls(controls):
    """Disable all controls."""
    for key in controls:
        controls[key].disabled = True


def shutdown_callback(
    arduino, daq_task, on_demand_controls
):
    # Disable controls
    disable_controls(on_demand_controls)

    
    arduino.write(bytes([ON_REQUEST]))

    # Stop DAQ async task
    daq_task.cancel()

    # Disconnect from Arduino
    arduino.close()

Now we build the app.

In [16]:
def spectrophotometer_app(
    arduino, on_demand_data, growth_data, blank_data, daq_task, rollover=400, stream_plot_delay=90,
):
    def _app(doc):
        # Plots
        p_on_demand, on_demand_source, on_demand_phantom_source, p_growth, growth_source, growth_phantom_source = plot("on demand")
        

        # Controls
        on_demand_controls = controls()

        # Shut down
        shutdown_button = bokeh.models.Button(
            label="shut down", button_type="danger", width=100
        )

        # Layouts
        on_demand_layout = layout(p_on_demand, p_growth, on_demand_controls)
        

        # Shut down layout
        shutdown_layout = bokeh.layouts.row(
            bokeh.models.Spacer(width=675), shutdown_button
        )

        app_layout = bokeh.layouts.column(
            on_demand_layout, shutdown_layout
        )

            
        def _on_demand_blank_callback(event=None):
            blank_callback(arduino, on_demand_controls)
        
        def _on_demand_reset_blank_callback(event=None):
            reset_blank_callback(arduino, on_demand_controls)
        
        def _acquire_callback(event=None):
            acquire_callback(
                arduino,
                blank_data,
                on_demand_source,
                on_demand_phantom_source,
                growth_source,
                growth_phantom_source,
                rollover,
            )
            
        

        def _on_demand_reset_callback(event=None):
            reset_data_callback(
                on_demand_data,
                growth_data,
                on_demand_source,
                growth_source,
                on_demand_phantom_source,
                growth_phantom_source,
                on_demand_controls,
            )

        def _on_demand_save_callback(event=None):
            save_callback(on_demand_data, growth_data, on_demand_controls)

        def _shutdown_callback(event=None):
            shutdown_callback(
                arduino, daq_task, on_demand_controls
            )
            
        

        # Link callbacks
        on_demand_controls["blank"].on_click(_on_demand_blank_callback)
        on_demand_controls["reset_blank"].on_click(_on_demand_reset_blank_callback)
        on_demand_controls["acquire"].on_click(_acquire_callback)
        on_demand_controls["reset_data"].on_click(_on_demand_reset_callback)
        on_demand_controls["save"].on_click(_on_demand_save_callback)
        shutdown_button.on_click(_shutdown_callback)

        # Add the layout to the app
        doc.add_root(app_layout)

    return _app

### Demonstration of incubating yeast

I let a few granules of yeast incubate in broth overnight. The following morning, I diluted the mixture 2 to 43 in fresh broth medium. I then measured the blank of just the broth medium. I measured in 30 minute intervals the first few measurements of the diluted yeast mixture. After 3 hours, I recorded OD measurements every 15-30 minutes until about hour 13. The results from the bokeh app can be found below.

In [17]:
bokeh.io.show(
    spectrophotometer_app(arduino, on_demand_data, growth_data, blank_data, daq_task),
    notebook_url=notebook_url,
)

Below we can find a screenshot from a completed run of growing yeast over ~12 hours.

![app_pic](Demonstration.png)

### Assessment

- $OD_{blank} - OD_{sample}$ must be **greater** than 0, or else the app will crash and need to be restarted. This is because it will not be able to plot a point on the growth curve.
- You are able to reset the blank dictionary, but the app will still function if you never clear previous measurements of blanks since it will only call on the latest version of the blank
- If you have to restart the app, you have to click reset data to clear any previous sample measurements. Or else they will just be appended to the existing dictionary.
- You are able to save time, OD, and ln(OD) to a csv file if you wish to perform analysis outside of the notebook
- It might be a good thing to add another Div widget to spit out the value of the OD being read when you click "acquire sample measurement" button so you know the value of the OD without looking at the LCD on the Arduino breadboard set up.

## Section 5: Analysis of data

In [114]:
"""Build a plot of optical density vs time data"""
# Set up plot area
p = bokeh.plotting.figure(
    frame_width=500,
    frame_height=175,
    x_axis_label="time (s)",
    y_axis_label="optical density (OD)",
    title="Optical density measurements",
    y_range=[-0.1, 1],
    toolbar_location="above",
)


# Defined the data source
source_demand = pd.DataFrame(on_demand_data)

# Use dots for on-demand
p.circle(source=source_demand, x="t", y="OD")


"""Build a plot of ln(optical density) vs time data"""
# Set up plot area
p1 = bokeh.plotting.figure(
    frame_width=500,
    frame_height=175,
    x_axis_label="time (h)",
    y_axis_label="lnOD600 (OD)",
    title="Growth curve",
    y_range=[-6.0, 0.5],
    toolbar_location="above",
)


# Defined the data source
source_growth = pd.DataFrame(growth_data)

# Use dots for on-demand
p1.circle(source=source_growth, x="t", y="lnOD")

bokeh.io.show(bokeh.layouts.column(p,p1))


We can see an exponential growth phase after a few hours of initial incubation. This becomes linear on the log plot. This is what we expect so that is good to see.

#### Performing linear regression to find the growth rate and doubling time.

In [47]:
from sklearn.linear_model import LinearRegression

In [115]:
# Get rid of first 3 points since they are not in the exponential growth curve
x = source_growth.t.to_numpy()[3:,].reshape((-1,1))
y = source_growth.lnOD.to_numpy()[3:,]

model = LinearRegression().fit(x,y)

print('coefficient of determination: ', model.score(x,y))
print('intercept:', model.intercept_)
print('slope:', float(model.coef_))

coefficient of determination:  0.9800465094538346
intercept: -3.853821996893773
slope: 0.31477862898383513


Now I want to visualize the plots above with the linear regression implemented.

In [118]:
"""Build a plot of optical density vs time data"""
# Set up plot area
p = bokeh.plotting.figure(
    frame_width=500,
    frame_height=175,
    x_axis_label="time (s)",
    y_axis_label="optical density (OD)",
    title="Optical density measurements",
    y_range=[-0.1, 1],
    toolbar_location="above",
)


# Defined the data source
source_demand = pd.DataFrame(on_demand_data)

# Use dots for on-demand
p.circle(source=source_demand, x="t", y="OD")

# Equation from linear regression
x_linreg = np.linspace(4,12,100)
y_linreg = float(model.coef_)*x_linreg+float(model.intercept_)

# Converting to exponential form
x_exp = x_linreg * 3600
y_exp = np.exp(float(model.coef_) * x_linreg + float(model.intercept_))

p.line(x_exp, y_exp, line_width=2, color='red')


"""Build a plot of ln(optical density) vs time data"""
# Set up plot area
p1 = bokeh.plotting.figure(
    frame_width=500,
    frame_height=175,
    x_axis_label="time (h)",
    y_axis_label="lnOD600 (OD)",
    title="Growth curve",
    y_range=[-6.0, 0.5],
    toolbar_location="above",
)


# Defined the data source
source_growth = pd.DataFrame(growth_data)

# Use dots for on-demand
p1.circle(source=source_growth, x="t", y="lnOD")

p1.line(x_linreg, y_linreg, line_width=2, color="red")

bokeh.io.show(bokeh.layouts.column(p,p1))


ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'file://';                       use --allow-websocket-origin= or set BOKEH_ALLOW_WS_ORIGIN= to permit this; currently we allow origins {'localhost:8888'}
ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'file://';                       use --allow-websocket-origin= or set BOKEH_ALLOW_WS_ORIGIN= to permit this; currently we allow origins {'localhost:8888'}
ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'event_name': 'button_click', 'event_values': {}}}], 'references': []} 
 error: IndexError('list index out of range')
Traceback (most recent call last):
  File "/Users/krystinbrown/opt/anaconda3/lib/python3.8/site-packages/bokeh/server/protocol_handler.py", line 90, in handle
    work = await handler(message, connection)
  File "/Users/krystinbrown/opt/anaconda3/lib/python3

In [117]:
# Find doubling time (hours)
print("Doubling time (hours): ", np.log(2)/float(model.coef_))

Doubling time (hours):  2.2020147390486935


To find the doubling time for yeast, we can use the slope of the cumulative curve, on the log scale. I found the slope by fitting a linear regression line to the data in which exponential growth was occurring. The doubling time is then found by $\frac{ln(2)}{k}=\frac{ln(2)}{0.3147}=2.20 \space hours$. However, the doubling time of yeast is actually 90 minutes or 1.5 hours. Therefore, my doubling time is about 44 minutes slower than what is expected. This could be due to a number of experimental errors. 

A few experimental variables to consider is:
- Not quite the optimal conditions for yeast growth. If so, the yeast could take longer to double.
- Yeast was not properly mixed with the broth. Every so often I shook the falcon tube with the solution. If yeast was not properly incorporated with the broth, growth could slow.
- Contamination. I used the same pipette and the same cuvette to run all my measurements. After I was done measuring a solution, I poured it back into the falcon tube with the rest of the mixture. These fluctuations are hard to pinpoint.
- OD measurements may not have been as sensitive as desirable to notice small changes. This could be due the experimental set up with the box and straws.
- I fell asleep from the end of hour 1 to the beginning of hour 5. I may have missed some crucial measurements in that beginning incubation period. If I did miss the beginning of the exponential growth curve, the slope of the log plot could be lower or higher than actual growth.

## Section 6: Suggestions for the next design phase

There are many things to consider in the next phase of design. First and foremost is a stable spectrophotometer setup. Perhaps if one was able to 3D print the setup that I made, but with more accurate angles, length between straws, etc. Most importantly, you want to be able to move a cuvette into place without knocking around the apertures or changing the bottom blank value. When I move my setup, I have to be pretty cautious, so it could be beneficial to somehow place the Arduino, breadboard, and spectrophotometer all the same base. As for the user interface, I would add text to output the OD value when it is acquired, similar to the blank value reading.